In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
Independent=dataset[['User ID','Gender_Male','Age','EstimatedSalary']]
Independent

,User ID,Gender_Male,Age,EstimatedSalary
0,15624510,1,19,19000
1,15810944,1,35,20000
2,15668575,0,26,43000
3,15603246,0,27,57000
4,15804002,1,19,76000
...,...,...,...,...
395,15691863,0,46,41000
396,15706071,1,51,23000
397,15654296,0,50,20000
398,15755018,1,36,33000


In [7]:
Dependent=dataset[['Purchased']]
Dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(Independent,Dependent,test_size=0.30,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'weights':['uniform','distance'],'algorithm':['auto','ball_tree','kd_tree','brute']}
grid=GridSearchCV(KNeighborsClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


C:\Users\nandh\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [11]:
y_pred=grid.predict(x_test) #(.predict->defined function)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[74  5]
 [ 6 35]]


In [12]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.93      0.94      0.93        79
           1       0.88      0.85      0.86        41

    accuracy                           0.91       120
   macro avg       0.90      0.90      0.90       120
weighted avg       0.91      0.91      0.91       120



In [13]:
re=grid.cv_results_
grid_predictions = grid.predict(x_test) 
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("f1_macro value for {}:".format(grid.best_params_),f1_macro)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
table=pd.DataFrame.from_dict(re)
table

f1_macro value for {'algorithm': 'auto', 'weights': 'uniform'}: 0.9080557496700055


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018708,0.006044,0.026545,0.006003,auto,uniform,"{'algorithm': 'auto', 'weights': 'uniform'}",0.872432,0.855314,0.858503,0.910254,0.888158,0.876932,0.020315,1
1,0.023111,0.004882,0.023222,0.004523,auto,distance,"{'algorithm': 'auto', 'weights': 'distance'}",0.872432,0.855314,0.841398,0.892857,0.888158,0.870032,0.019452,5
2,0.018097,0.006394,0.022025,0.005520,ball_tree,uniform,"{'algorithm': 'ball_tree', 'weights': 'uniform'}",0.872432,0.855314,0.858503,0.910254,0.888158,0.876932,0.020315,1
3,0.015716,0.005417,0.013664,0.001991,ball_tree,distance,"{'algorithm': 'ball_tree', 'weights': 'distance'}",0.872432,0.855314,0.841398,0.892857,0.888158,0.870032,0.019452,5
4,0.011284,0.004421,0.022423,0.003692,kd_tree,uniform,"{'algorithm': 'kd_tree', 'weights': 'uniform'}",0.872432,0.855314,0.858503,0.910254,0.888158,0.876932,0.020315,1
5,0.009969,0.003586,0.013492,0.004714,kd_tree,distance,"{'algorithm': 'kd_tree', 'weights': 'distance'}",0.872432,0.855314,0.841398,0.892857,0.888158,0.870032,0.019452,5
6,0.006020,0.003969,0.276723,0.006549,brute,uniform,"{'algorithm': 'brute', 'weights': 'uniform'}",0.872432,0.855314,0.858503,0.910254,0.888158,0.876932,0.020315,1
7,0.004611,0.004963,0.265636,0.003439,brute,distance,"{'algorithm': 'brute', 'weights': 'distance'}",0.872432,0.855314,0.841398,0.892857,0.888158,0.870032,0.019452,5
